In [2]:
import re
import networkx as nx
from pathlib import Path
from networkx.drawing.nx_agraph import graphviz_layout

In [3]:
from flinspect.parse_forest import ParseForest

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
build_dirs = [
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/FMS/",
    "/glade/work/altuntas/turbo-stack/bin/flang_ptree/MOM6/"
]

parse_tree_paths = [f for dir in build_dirs for f in Path(dir).glob("*_ptree")]

In [6]:
pf = ParseForest(parse_tree_paths)

In [7]:
G = pf.get_module_dependency_graph()

Skipped 8 modules with unknown parse tree paths: ['netcdf', 'iso_c_binding', 'iso_fortran_env', 'mpi', 'netcdf_nf_data', 'netcdf_nf_interfaces', 'netcdf4_nf_interfaces', 'omp_lib']


In [8]:
len(G), len(G.edges)

(398, 3105)

In [9]:
pos = nx.nx_agraph.graphviz_layout(G, prog='fdp')

In [10]:
# Determine source paths via the path_names files in build dirs
# This will be used to color nodes and edges based on where modules are located
path_names = {}
for build_dir in build_dirs:
    path_names_file = Path(build_dir)/"path_names"
    with open(path_names_file) as f:
        for line in f:
            src_file_path = Path(line.strip())
            path_names[src_file_path.stem.lower()] = src_file_path.as_posix()

In [11]:
def get_color(G, node):
    source_name = G.nodes[node].get('source_name','')
    source_path = path_names.get(source_name.lower(),'')
    
    node_color, edge_color = "grey", "lightgrey"
    if "/config_src/infra" in source_path:
        node_color, edge_color = "darkred", "red"    
    elif "/MARBL/" in source_path:
        node_color, edge_color = "purple", "orchid"
    elif "/CVMix-src/" in source_path:
        node_color, edge_color = "chocolate", "sandybrown"
    elif "/GSW-Fortran/" in source_path:
        node_color, edge_color = "royalblue", "cornflowerblue"
    elif "/MOM6/" in source_path:
        node_color, edge_color = "lightseagreen", "turquoise" # for core MOM6        
    elif "/FMS/" in source_path:
        node_color, edge_color = "orange", "gold"
        
    return node_color, edge_color

In [12]:
from pyvis.network import Network

In [13]:
net = Network(notebook=True, directed=True, height='800px', cdn_resources='in_line')

net.set_options("""
var options = {
  "physics": { "enabled": false },
  "nodes": { "shape": "dot", "size": 10 },
  "edges": { "arrows": { "to": { "enabled": true, "scaleFactor": 0.3 }    } }
}
""")

# Add nodes with positions
for node in G.nodes():

    node_color, _ = get_color(G, node)

    x, y = pos[node]
    net.add_node(str(node), color=node_color, label=str(node), x=x, y=-y, fixed=True)  # y flipped for browser coord system

# Add edges
for source, target in G.edges():
    _, edge_color = get_color(G, source)
    net.add_edge(str(source), str(target), color=edge_color)

In [14]:
net.show("module_dependency.html") 

module_dependency.html
